In [1]:
from scrap import Scraper

In [2]:
url = "https://psopk.com/en/our-group/our-vision-mission-values"
scrp = Scraper(url)

In [3]:
scrp.scrape_website()

In [4]:
from RAG_QnA import RAG_Model

In [5]:
rag_model = RAG_Model()

2024-07-29 10:06:22.107371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 10:06:22.305763: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 10:06:22.352194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 10:06:25.005308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [6]:
print(rag_model.generateResponse(prompt="What is dates of entry test in numl?"))

Number of requested results 50 is greater than number of elements in index 6, updating n_results = 6


1. The dates for the entry test in NUML have not been specified in the provided advertisement for Fall 2024.
        2. Eligible candidates will receive their admit cards for the entry test after document verification and eligibility check as per the advertisement.
        3. For queries, please call at 051-9265118, 051-9265119 or email at admissions@numl.edu.pk.
        4. The registration for admission to various courses in Fall 2024 has been extended till July 05, 2024 (03:00 P.M).
        5. Start of classes for Fall 2024 is scheduled for September 09, 2024.
        6. It is advisable to regularly check the official website or contact the university for the latest updates regarding the entry test dates.
        7. Thanks for asking!</s>
